### State generation and swap product


In [1]:
%load_ext autoreload
%autoreload 2

import cirq
import numpy as np
import sys

sys.path.append("C:/Users/Alj/CODING/projects/Quantum_Computing")
from state_generation import StateMaker, sphere_samples

In [2]:
sm = StateMaker()
sm.set_default_operator()
n = 3

In [3]:

target = sphere_samples(n_dim=2**n, n_samples=1, subset="non-negative").flatten()
target

array([0.20418631, 0.57639286, 0.14528036, 0.6710828 , 0.11666738,
       0.29039587, 0.05697107, 0.23115789])

In [4]:
t1 = np.array([1,2,3,4,5,6,7,8])
t2 = np.array([5,5,5,5,0,0,0,0])

res1 = sm.approx_target_state(t1)
res2 = sm.approx_target_state(t2)

arguments of state_generator: {phi_1, phi_4, phi_5, phi_0, phi_2, phi_3}
target vector was scaled by  0.07001400420140048  to be unit vector
arguments of state_generator: {phi_1, phi_4, phi_5, phi_0, phi_2, phi_3}
target vector was scaled by  0.1  to be unit vector


In [5]:
true_dotprod = np.abs(np.vdot(res1["state_approx"], res2["state_approx"]))
print("vectors have dot product of ", true_dotprod)

vectors have dot product of  0.35009626115339393


In [7]:
#
c1 = sm.get_circ_of_default_op(res1["arg_angles"])
c2 = sm.get_circ_of_default_op(res2["arg_angles"])

# shift qbits of one circuit to avoid duplicate qbits in swap test
def shift_qbits(qbit):
    row = qbit.row
    return cirq.GridQubit(row+3, 0)
c1 = c1.transform_qubits(shift_qbits)

In [7]:
#
c1

(3, 0): ───Ry(0.134π)───@───@───Ry(-1.89π)───@───Ry(-1.95π)───@───@───Ry(0.467π)───
                        │   │                │                │   │
(4, 0): ───Ry(0.627π)───X───┼────────────────@────────────────@───┼────────────────
                            │                │                │   │
(5, 0): ───Ry(0.562π)───────X────────────────X────────────────X───X────────────────

In [9]:
# get sorted list from frozen set, sorted by row
qbits_c1 = sorted(list(c1.all_qubits()), key=lambda x: x.row)
qbits_c2 = sorted(list(c2.all_qubits()), key=lambda x: x.row)


In [10]:
# make swaptest
output_qubit = cirq.GridQubit(6, 0)

In [11]:
n_qubits = 3
c = cirq.Circuit()
# initialize qubits
c.append(cirq.H(output_qubit))
c.append(c1)
c.append(c2)
# swap pairs
c.append([cirq.CSWAP(output_qubit, qbits_c1[i], qbits_c2[i]) 
          for i in range(n_qubits)
         ])
c.append(cirq.H(output_qubit))
c.append(cirq.measure(output_qubit))
c

(0, 0): ─────────────────────────────────────────────────────────────────────────────Ry(-1.92π)───@───@───Ry(1.78π)───@───Ry(-1.15π)───@───@───Ry(-0.71π)───×───────────────────
                                                                                                  │   │               │                │   │                │
(1, 0): ─────────────────────────────────────────────────────────────────────────────Ry(0.5π)─────X───┼───────────────@────────────────@───┼────────────────┼───×───────────────
                                                                                                      │               │                │   │                │   │
(2, 0): ─────────────────────────────────────────────────────────────────────────────Ry(-1.5π)────────X───────────────X────────────────X───X────────────────┼───┼───×───────────
                                                                                                                                                            │   │   │
(3, 0): ───────Ry(0.134π)───@───@───Ry(1.11π)───@───Ry(0.054π)───@───@───Ry(1.47π)──────────────────────────────────────────────────────────────────────────×───┼───┼───────────
                            │   │               │                │   │                                                                                      │   │   │
(4, 0): ───────Ry(0.627π)───X───┼───────────────@────────────────@───┼──────────────────────────────────────────────────────────────────────────────────────┼───×───┼───────────
                                │               │                │   │                                                                                      │   │   │
(5, 0): ───────Ry(-1.56π)───────X───────────────X────────────────X───X──────────────────────────────────────────────────────────────────────────────────────┼───┼───×───────────
                                                                                                                                                            │   │   │
(6, 0): ───H────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────@───@───@───H───M───

In [12]:
#
n_csamples = 100000
simulator = cirq.Simulator()
result = simulator.run(c,repetitions=n_csamples)
measurement_mean = result.data.mean().values[0]
P0 = np.maximum((1 - measurement_mean),0.5)
print("P0", P0)
estim = (2*(P0 - 0.5))**0.5
print("vectors have dot product of ", true_dotprod)
print("estimated dot product: ", estim)

P0 0.5613600000000001
vectors have dot product of  0.35009626115339393
estimated dot product:  0.3503141447329813


In [58]:
#


In [20]:
#